In [ ]:
import pandas as pd
from spdynamodb import DynamoTable
from time import sleep
import json
import time
from decimal import Decimal

In [ ]:
#dt = DynamoTable(profile_name='my-profile')
dt=DynamoTable()
try:
    dt.select_table('FooBarTable')
    print(dt)
except:
    dt.create_table(
        table_name='FooBarTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
    )

### Create 3 global secondary indexes

In [ ]:
# 1st Global Secondary Index
dt.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    sort_index="GSI1-SK",
    sort_type="S",
    i_name="GSI1"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("1st global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        sleep(30)
end = time.time()
minute = (end - start) / 60
print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

# 2nd Global Secondary Index
dt.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    sort_index="GSI2-SK",
    sort_type="S",
    i_name="GSI2"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("2nd global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        sleep(30)
end = time.time()
minute = (end - start) / 60
print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

# 3rd Global Secondary Index
dt.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    sort_index="GSI3-SK",
    sort_type="N",
    i_name="GSI3"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("3rd global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        sleep(30)
end = time.time()
minute = (end - start) / 60
print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

### Users Table

In [ ]:
df_users = pd.read_csv('DesigningDynamoDBTable-1/users.csv')

df_users['PK'] = 'USER#' + df_users['email'].astype(str)
df_users['SK'] = 'USER#' + df_users['email'].astype(str)
df_users.head()

### Orders Table

In [ ]:
df_orders = pd.read_csv('DesigningDynamoDBTable-1/orders.csv')

df_orders['PK'] = 'ORDER#' + df_orders['orderId'].astype(str)
df_orders['SK'] = 'ORDER#' + df_orders['orderId'].astype(str)
df_orders['GSI1-PK'] = 'USER#' + df_orders['userId'].astype(str)
df_orders['GSI1-SK'] = 'OSTATUS#' + df_orders['status'].astype(str)
df_orders.head()

### Order details Table

In [ ]:
df_order_details = pd.read_csv('DesigningDynamoDBTable-1/order_items.csv')

df_order_details['orderId'] = df_order_details['orderId'] + 1000
df_order_details['PK'] = 'ORDER#' + df_order_details['orderId'].astype(str)
df_order_details['SK'] = 'PRODUCT#' + df_order_details['productId'].astype(str)
df_order_details.head()

### Products Table

In [ ]:
df_products = pd.read_csv('DesigningDynamoDBTable-1/products.csv')
values = [100,101,102,200,201,202,301,400,401,402,500,501,502,601,701,702,801,802,900,902]
df_products['productId'] = values
df_products['PK'] = 'PRODUCT#' + df_products['productId'].astype(str)
df_products['SK'] = 'PRODUCT#' + df_products['productId'].astype(str)
df_products['GSI1-PK'] = 'PRODUCT'
df_products['GSI1-SK'] = 'PNAME#' + df_products['name'].astype(str).str.lower()
df_products['GSI2-SK'] =  df_products['category'].astype(str).str.lower()
df_products['GSI3-SK'] =  df_products['price']
df_products.head()

### Add data to DynamoDB

In [ ]:
dt.batch_pandas(dataframe=df_users)
dt.batch_pandas(dataframe=df_orders)
dt.batch_pandas(dataframe=df_order_details)
dt.batch_pandas(dataframe=df_products)

### Querying the table

In [ ]:
dt.query(pk_value="ORDER#1009", to_pandas=True)

In [ ]:
dt.table.put_item(
    Item={
        'PK': 'ORDER#1011',
        'SK': 'ORDER#1011',
        'purchaseDate': '2020-03-14T15:32:52'
    }
)
dt.table.put_item(
    Item={
        'PK': 'ORDER#1011',
        'SK': 'ORDER#1012',
        'purchaseDate': '2020-03-15T15:32:52'
    }
)
dt.table.put_item(
    Item={
        'PK': 'ORDER#1011',
        'SK': 'ORDER#1013',
        'purchaseDate': '2020-03-17T15:32:52'
    }
)
dt.table.put_item(
    Item={
        'PK': 'ORDER#1011',
        'SK': 'ORDER#1014',
        'purchaseDate': '2020-03-19T15:32:52'
    }
)

In [ ]:
query = '2015-01-01T00:00:00_2023-03-17T23:59:59'
dt.query(pk_value='ORDER#1011', sk_value=query, consumed_capacity='TOTAL', index_name='GSI4', to_pandas=True)

In [ ]:
dt.query(pk_value='PRODUCT', sk_value="==59.99", consumed_capacity='TOTAL', index_name='GSI3', to_pandas=True)

In [ ]:
dt.query(pk_value='PRODUCT#301', sk_value="PRODUCT*", consumed_capacity='TOTAL', to_pandas=True)